In [1]:
import os, sys, json
sys.path.append("/home/tristan/projects/LLM/AutoCog")
from autocog.language.sta.frontend import frontend
from autocog.utility.pynb import wrap_graphviz, display

## Single prompt and _compact_

**Note**:
 - `text` is implicitly `text<INF>` (meaning no limit on number of tokens) 
   - **but** can only be used if target of a channel

In [2]:
tree = frontend("""
prompt mmlu_main {
    define default_number_of_choices=4;
    argument N=$default_number_of_choices;
    is {
        topic is text;
        question is text;
        choices[$N] is {
            value is text;
            correct is bool;
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}
""")
wrap_graphviz(tree.toGraphViz())

## Reusable data-structure

**Note**:
 - showing shorthand for single element `return`

In [3]:
tree = frontend("""
struct a_choice {
    is {
        value   is text;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text;
        question   is text;
        choices[4] is a_choice;
        answer     is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
""")

## Flow between prompts and formats

**Note**:
 - annotation with f-exp `f"{var}"`
 - global `flow` declares possible entry point
   - entry point defaults to `main`
   - if not precised the first prompt defaults to be `main`

In [4]:
tree = frontend("""
format sentence {
    argument N=30;
    is text<$N>;
    annotate f"A grammatically correct sentence made of at most {N} tokens.";
}

prompt mmlu_main {
    define K=50;
    is {
        topic       is text;
        question    is text;
        hyphothesis is sentence<$K>;
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
    }
    flow {
        to mmlu_choose;
    }
}

prompt mmlu_choose {
    is {
        topic       is text;
        question    is text;
        hyphothesis is sentence;
        choices[4]  is text;
        answer      is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
        to .hyphothesis   from mmlu_main.hyphothesis;
    }
    return .answer;
}

flow {
    to mmlu_main as "main";
}
""")

## Iterations

**Note**:
 - use of `as` in both `flow` and `return`, when prompt ends LLM will have to choose between `retry` and `ready`
   - without `as` choice would have been `mmlu_main` or `return`
 - `[5]` in the flow section limits the trip count on that branch ensuring termination

In [5]:
tree = frontend("""
prompt mmlu_main {
    is {
        topic         is text;
        question      is text;
        choices[4]    is text;
        previous[0:1] is text<20>;
        answer        is repeat(.choices.value);
        reflect       is text<20>;
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
        to .previous      from .reflect;
    }
    flow {
        to mmlu_main[5] as "retry";
    }
    return {
        as "ready";
        from .answer;
    }
}
""")

## Calls

**Note**:
 - 

In [6]:
tree = frontend("""
prompt mmlu_main {
    is {
        topic is text;
        question is text;
        choices[4] is {
            value is text;
            correct is bool;
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
        to .choices call {
            entry mmlu_annot;
            kwarg topic    from ?topic;
            kwarg question from ?question;
            kwarg choice   map  ?choices;
            bind choice as value;
        }
    }
    return {
        from .answer;
    }
}

prompt mmlu_annot {
    is {
        topic    is text;
        question is text;
        choice   is text;
        correct  is bool;
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
        to .choice   from ?choice;
    }
    return {
        from .choice;
        from .correct;
    }
}
""")

IncompleteParseError: Rule 'program' matched in its entirety, but it didn't consume all the text. The non-matching portion of the text begins with 'prompt mmlu_main {
 ' (line 2, column 1).